In [31]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
#%run ./bayseian_search//plotters.py
%matplotlib notebook

In [2]:
import numpy as np
import sklearn.gaussian_process as gp

from scipy.stats import norm
from scipy.optimize import minimize

def expected_improvement(x, gaussian_process, evaluated_loss, greater_is_better=False, n_params=1):
    """ expected_improvement

    Expected improvement acquisition function.

    Arguments:
    ----------
        x: array-like, shape = [n_samples, n_hyperparams]
            The point for which the expected improvement needs to be computed.
        gaussian_process: GaussianProcessRegressor object.
            Gaussian process trained on previously evaluated hyperparameters.
        evaluated_loss: Numpy array.
            Numpy array that contains the values off the loss function for the previously
            evaluated hyperparameters.
        greater_is_better: Boolean.
            Boolean flag that indicates whether the loss function is to be maximised or minimised.
        n_params: int.
            Dimension of the hyperparameter space.
    """
    x_to_predict = x.reshape(-1, n_params)

    mu, sigma = gaussian_process.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    # In case sigma equals zero
    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        expected_improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)
        expected_improvement[sigma == 0.0] == 0.0

    return -1 * expected_improvement

In [3]:
def sample_next_hyperparameter(acquisition_func, gaussian_process, evaluated_loss, greater_is_better=False,
                               bounds=(0, 10), n_restarts=25):
    """ sample_next_hyperparameter

    Proposes the next hyperparameter to sample the loss function for.

    Arguments:
    ----------
        acquisition_func: function.
            Acquisition function to optimise.
        gaussian_process: GaussianProcessRegressor object.
            Gaussian process trained on previously evaluated hyperparameters.
        evaluated_loss: array-like, shape = [n_obs,]
            Numpy array that contains the values off the loss function for the previously
            evaluated hyperparameters.
        greater_is_better: Boolean.
            Boolean flag that indicates whether the loss function is to be maximised or minimised.
        bounds: Tuple.
            Bounds for the L-BFGS optimiser.
        n_restarts: integer.
            Number of times to run the minimiser with different starting points.

    """
    best_x = None
    best_acquisition_value = 1
    n_params = bounds.shape[0]

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, n_params)):

        res = minimize(fun=acquisition_func,
                       x0=starting_point.reshape(1, -1),
                       bounds=bounds,
                       method='L-BFGS-B',
                       args=(gaussian_process, evaluated_loss, greater_is_better, n_params))

        if res.fun < best_acquisition_value:
            best_acquisition_value = res.fun
            best_x = res.x

    return best_x

In [4]:
def bayesian_optimisation(n_iters, sample_loss, bounds, x0=None, n_pre_samples=5,
                          gp_params=None, random_search=False, alpha=1e-5, epsilon=1e-7):
    """ bayesian_optimisation

    Uses Gaussian Processes to optimise the loss function `sample_loss`.

    Arguments:
    ----------
        n_iters: integer.
            Number of iterations to run the search algorithm.
        sample_loss: function.
            Function to be optimised.
        bounds: array-like, shape = [n_params, 2].
            Lower and upper bounds on the parameters of the function `sample_loss`.
        x0: array-like, shape = [n_pre_samples, n_params].
            Array of initial points to sample the loss function for. If None, randomly
            samples from the loss function.
        n_pre_samples: integer.
            If x0 is None, samples `n_pre_samples` initial points from the loss function.
        gp_params: dictionary.
            Dictionary of parameters to pass on to the underlying Gaussian Process.
        random_search: integer.
            Flag that indicates whether to perform random search or L-BFGS-B optimisation
            over the acquisition function.
        alpha: double.
            Variance of the error term of the GP.
        epsilon: double.
            Precision tolerance for floats.
    """

    x_list = []
    y_list = []

    n_params = bounds.shape[0]

    if x0 is None:
        for params in np.random.uniform(bounds[:, 0], bounds[:, 1], (n_pre_samples, bounds.shape[0])):
            x_list.append(params)
            y_list.append(sample_loss(params))
    else:
        for params in x0:
            x_list.append(params)
            y_list.append(sample_loss(params))

    xp = np.array(x_list)
    yp = np.array(y_list)

    # Create the GP
    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)

    for n in range(n_iters):

        model.fit(xp, yp)

        # Sample next hyperparameter
        if random_search:
            x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
            ei = -1 * expected_improvement(x_random, model, yp, greater_is_better=True, n_params=n_params)
            next_sample = x_random[np.argmax(ei), :]
        else:
            next_sample = sample_next_hyperparameter(expected_improvement, model, yp, greater_is_better=True, bounds=bounds, n_restarts=100)

        # Duplicates will break the GP. In case of a duplicate, we will randomly sample a next query point.
        if np.any(np.abs(next_sample - xp) <= epsilon):
            next_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], bounds.shape[0])

        # Sample loss for new set of parameters
        cv_score = sample_loss(next_sample)

        # Update lists
        x_list.append(next_sample)
        y_list.append(cv_score)

        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

    return xp, yp

In [5]:
summed_sen_df = pickle.load( open( "sen_dataframes/sen_data_summed.p", "rb" ) )  
not_summed_sen_df = pickle.load( open( "sen_dataframes/sen_data.p", "rb" ) )  

In [6]:
from sklearn.model_selection import train_test_split 
def generate_training_data(dataframe, predicting):
    '''
    If you want to retrain algorithm with diffrent hyperparameters of train a new model altogether,
    this function will take the dataframe and return vectors which are used to train and evaluate the models.

    X_all and y_all should only be used to train a regressor that 
    will make a prediction on a case outside of the Whisper catalogue.

    Parameters
    ----------
        dataframe : Pandas DataFrame
        Should contain all of the cases that will be trained and tested upon.

        predicting : string
        Either 'bias' or 'keig_meas' depending on what you're predicting
    Returns
    -----------
    X_train, X_test, X_all, matrix of shape = [n_samples, n_features]
    y_train, y_test, y_all, array, shape = [n_samples, 1]
    '''
    train_set, test_set = train_test_split(dataframe, test_size=0.2, random_state=42)

    X_all = np.vstack(dataframe['s'].values)
#    X_all = np.insert(X_all, 0, dataframe['keig_siml'], axis =1)

    y_all = np.asarray(dataframe[predicting].values).reshape(-1,1)

    X_train = np.vstack(train_set['s'].values)
    X_test = np.vstack(test_set['s'].values)

#    X_train = np.insert(X_train, 0, train_set['keig_siml'], axis =1)
#    X_test = np.insert(X_test, 0, test_set['keig_siml'], axis =1)

    y_train = np.asarray(train_set[predicting].values).reshape(-1,1)
    y_test = np.asarray(test_set[predicting].values).reshape(-1,1)
    
    return X_train, X_test, y_train, y_test, X_all, y_all

X_train, X_test, y_train, y_test, X_all, y_all = generate_training_data(summed_sen_df, 'bias')

In [10]:
from sklearn.model_selection import cross_val_scorehow 
from sklearn.ensemble import RandomForestRegressor

def sample_loss(params):
    return cross_val_score(RandomForestRegressor(n_estimators = int(params[0]), max_features = int(params[1]),  
                            random_state=12345),X=X_train, y=y_train.ravel(), #max_depth = int(params[2])
                           cv=3, scoring = 'neg_mean_squared_error').mean()

In [11]:
def find_lowest_error(x,y):
    print(x[np.array(y).argmax()], y.max())

In [12]:
# n_estimators, max_features, max_depth
bounds = np.array([[1000, 10000], [10, 200]])

xp, yp = bayesian_optimisation(n_iters=25, 
                               sample_loss=sample_loss, 
                               bounds=bounds,
                               n_pre_samples=4)

In [13]:
find_lowest_error(xp,yp)

[ 6912.01935275   114.52004763] -1.58532024345e-05


In [ ]:
# n_estimators, max_features, max_depth
bounds = np.array([[100, 10000], [1, 200]])

xp1, yp1 = bayesian_optimisation(n_iters=100, 
                               sample_loss=sample_loss, 
                               bounds=bounds,
                               n_pre_samples=4)

In [ ]:
find_lowest_error(xp1,yp1)

In [ ]:
fig = plt.figure()
X = xp1[:,0]
Y = xp1[:,1]
ax = fig.gca(projection='3d')
ax.scatter(X, Y, yp1, s=50, c = 'r', alpha = 1.0, marker = 'o')
# ax.set_xlabel(r'Aspect Ratio$(\frac{diameter}{thckness})$')
# ax.set_ylabel('Temperature [K]')
# ax.set_zlabel(r' $(\frac{Calculated}{Actual}) \alpha$')
# ax.set_title(r'Deviation of $\alpha$ vs. Temeprature & Aspect Ratio')
# ax.tick_params(axis='y', width=10, labelsize=10, pad=0)
# plt.savefig('Analytical.jpg', dpi=1000)	
plt.show()